# Clustering Neighbourhoods in Toronto¶


## Part 1 - Converting Wikipedia html table into a DataFrame¶


### Pre-processing¶


In [39]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#print(soup.prettify())


### extracting raw table (from webpage)

In [41]:
# extracting the raw table inside that webpage
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

# extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

### Processing-part-2: integrating Postal codes with more than 1 neighbour¶



In [42]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


### Post-processing: creating an appropriate Pandas Dataframe¶


In [43]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M2H,North York,Hillcrest Village
1,M4T,Central Toronto,Moore Park
2,M1X,Scarborough,Upper Rouge
3,M4K,East Toronto,Riverdale
4,M4B,East York,"Woodbine Gardens, Parkview Hill"
5,M5C,Downtown Toronto,St. James Town
6,M3K,North York,CFB Toronto
7,M6H,West Toronto,Dovercourt Village
8,M6P,West Toronto,High Park
9,M5X,Downtown Toronto,"First Canadian Place, Underground city"


In [44]:
df_toronto.shape


(77, 3)

####  Dropping any row where Borough is not assigned¶


In [6]:
neighbor_df = neighbor_df[neighbor_df.Borough != 'Not assigned']

neighbor_df.reset_index(drop=True, inplace=True)

neighbor_df.head(20)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [ ]:
DT_venues.groupby('Neighborhood').count()


In [ ]:
print('There are {} uniques categories.'.format(len(DT_venues['Venue Category'].unique())))


Analyze each neighborhood's venues¶


In [ ]:
# one hot encoding
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

In [ ]:
DT_onehot.shape


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [ ]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped

Let's print each neighborhood along with the top 5 most common venues¶


In [ ]:
num_top_venues = 5

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Put the top venues information into a dataframe¶


First, write a function to sort the venues in descending order



In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top 10 venues¶


In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

### Cluster the Neighborhoods of Downtown Toronto¶


In [ ]:
# run K-means to cluster the neighborhoods into 5 clusters

# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

### Create a new dataframe that inclues the cluster as well as the top 10 venues¶


In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

DT_merge = DT_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merge = DT_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merge.head() # check the last columns!

#### Examine Clusters¶


In [ ]:
DT_merge.loc[DT_merge['Cluster'] == 0, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]


In [ ]:
DT_merge.loc[DT_merge['Cluster'] == 1, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]


In [ ]:
DT_merge.loc[DT_merge['Cluster'] == 2, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]


In [ ]:
DT_merge.loc[DT_merge['Cluster'] == 3, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]


In [ ]:
DT_merge.loc[DT_merge['Cluster'] == 4, DT_merge.columns[[2] + list(range(5, DT_merge.shape[1]))]]
